In [ ]:
import cv2
import numpy as np
import os
import gradio as gr

In [30]:
def match(action, name, fingerprint_scan):
  result = fingerprint_scan
  x = "No matches found."

  if(action == "Enrollment"):
    dir = "./database/"
    filename = dir + name + '.jpg'
    cv2.imwrite(filename, fingerprint_scan)
    x = "Succesfully Registered as " + name
    sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(fingerprint_scan, None)
    cv2.drawKeypoints(fingerprint_scan, keypoints, fingerprint_scan)
    result = fingerprint_scan
    return result,x

  else:
    for file in os.listdir("./database/"):
      fingerprint_database_image = cv2.imread("./database/"+file)
      sift = cv2.xfeatures2d.SIFT_create()
      keypoints_1, descriptors_1 = sift.detectAndCompute(fingerprint_scan, None)
      keypoints_2, descriptors_2 = sift.detectAndCompute(fingerprint_database_image, None)
      matches = cv2.FlannBasedMatcher(dict(algorithm=1, trees=10), dict()).knnMatch(descriptors_1, descriptors_2, k=2)
      match_points = []
      for p, q in matches:
        if p.distance < 0.1*q.distance:
          match_points.append(p)
          keypoints = 0
          if len(keypoints_1) <= len(keypoints_2):
              keypoints = len(keypoints_1)                
          else:
              keypoints = len(keypoints_2)
              
          if (len(match_points) / keypoints)>0.95:
              result = cv2.drawMatches(fingerprint_scan, keypoints_1, fingerprint_database_image, keypoints_2, match_points, None) 
              a = len(match_points) / keypoints * 100
              x = "%.2f" % a + "% match with " + str(file)
              result = cv2.resize(result, None, fx=2.5, fy=2.5)
              break
    return result,x

In [31]:
iface = gr.Interface(fn=match,
                    title="Fingerprint Recognition System",  
                    inputs=[gr.inputs.Radio(["Enrollment", "Identification"],label="Action"),gr.inputs.Textbox(label="Name"), gr.inputs.Image()], 
                    outputs=[gr.outputs.Image(label="Flann-Based Matching Result on SIFT Features"), gr.outputs.Textbox(label="Match Percentage")], 
                    theme="compact",
                    allow_screenshot=False, 
                    allow_flagging=False)
iface.launch()

Running locally at: http://127.0.0.1:7873/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7873/', None)